In [55]:
import numpy as np
import torch
import torch.nn as nn
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data_path = 'HW1_data.csv'
data = np.genfromtxt(data_path, delimiter=',', skip_header=1)
Y = data[:,0]
X = data[:,1:]

Y = torch.tensor(Y)
X = torch.tensor(X)

In [56]:
n_samples, n_features = X.shape

In [58]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=1234)
scaler = StandardScaler()
X_Train = scaler.fit_transform(X_Train)
X_Test = scaler.transform(X_Test)


X_Train = torch.from_numpy(X_Train).float()
X_Test = torch.from_numpy(X_Test).float()


In [59]:
Y_Train = Y_Train.view(Y_Train.shape[0],1)
Y_Test = Y_Test.view(Y_Test.shape[0],1)

In [60]:
class PoissonRegression(nn.Module):
    def __init__(self, input_dim):
        super(PoissonRegression, self).__init__()
        # Linear layer (input_dim -> 1)
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        # Exponential function to ensure the output is positive
        return torch.exp(self.linear(x))
    


In [66]:
model = PoissonRegression(n_features)

In [67]:
def poisson_loss(y_predicted, y_true):
    return -torch.mean(y_true * torch.log(y_predicted) - y_predicted)

In [69]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [74]:
num_epochs = 10

for epoch in range(num_epochs):
    # forward pass and loss
    y_pred = model(X_Train)
    loss=poisson_loss(y_pred, Y_Train)
    # backward 
    loss.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    if epoch % 100 == 0:
        print(epoch, loss.item())

0 -0.6672607307449653


In [73]:
with torch.no_grad():
    y_pred = model(X_Test)
    y_pred_cls = y_pred.round()
    
    acc = y_pred_cls.eq(Y_Test).sum()/float(Y_Test.shape[0])
    print(f"Accuracy {acc}")

Accuracy 0.14499999582767487
